In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan.model import DATGAN
#from modules.datgan_preLSTM.model import DATGAN

#from modules.datgan_no_att.model import DATGAN
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/DATGAN_cpl/'

In [10]:
datgan = DATGAN(continuous_columns, max_epoch=10, steps_per_epoch=2000, 
                batch_size=200, output=output_folder, gpu=0)

In [11]:
datgan.fit(df, graph)

[0929 16:14:29 @model.py:928] Preprocessing the data!
[0929 16:14:30 @model.py:942] Preprocessed data have been saved!



[0929 16:14:31 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[0929 16:14:31 @model.py:318] Creating cell for age (in-edges: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0929 16:14:31 @registry.py:126] gen/LSTM/00/FC input: [200, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0929 16:14:31 @registry.py:134] gen/LSTM/00/FC output: [200, 100]
[0929 16:14:31 @registry.py:126] gen/LSTM/00/FC2 input: [200, 100]
[0929 16:14:31 @registry.py:134] gen/LSTM/00/FC2 output: [200, 1]
[0929 16:14:31 @registry.py:126] gen/LS

[0929 16:14:31 @registry.py:134] gen/LSTM/07/FC output: [200, 100]
[0929 16:14:31 @registry.py:126] gen/LSTM/07/FC2 input: [200, 100]
[0929 16:14:31 @registry.py:134] gen/LSTM/07/FC2 output: [200, 8]
[0929 16:14:31 @registry.py:126] gen/LSTM/07/FC3 input: [200, 8]
[0929 16:14:31 @registry.py:134] gen/LSTM/07/FC3 output: [200, 100]
[0929 16:14:31 @model.py:318] Creating cell for hh_income (in-edges: 2)
[0929 16:14:31 @registry.py:126] gen/LSTM/08/FC_inputs input: [200, 200]
[0929 16:14:31 @registry.py:134] gen/LSTM/08/FC_inputs output: [200, 100]
[0929 16:14:31 @registry.py:126] gen/LSTM/08/FC_attentions input: [200, 200]
[0929 16:14:31 @registry.py:134] gen/LSTM/08/FC_attentions output: [200, 100]
[0929 16:14:31 @registry.py:126] gen/LSTM/08/FC_lstm_state_0 input: [200, 200]
[0929 16:14:31 @registry.py:134] gen/LSTM/08/FC_lstm_state_0 output: [200, 100]
[0929 16:14:31 @registry.py:126] gen/LSTM/08/FC_lstm_state_1 input: [200, 200]
[0929 16:14:31 @registry.py:134] gen/LSTM/08/FC_lstm_st

[0929 16:14:32 @registry.py:126] gen/LSTM/21/FC_lstm_state_0 input: [200, 200]
[0929 16:14:32 @registry.py:134] gen/LSTM/21/FC_lstm_state_0 output: [200, 100]
[0929 16:14:32 @registry.py:126] gen/LSTM/21/FC_lstm_state_1 input: [200, 200]
[0929 16:14:32 @registry.py:134] gen/LSTM/21/FC_lstm_state_1 output: [200, 100]
[0929 16:14:32 @registry.py:126] gen/LSTM/22/FC input: [200, 100]
[0929 16:14:32 @registry.py:134] gen/LSTM/22/FC output: [200, 100]
[0929 16:14:32 @registry.py:126] gen/LSTM/22/FC2 input: [200, 100]
[0929 16:14:32 @registry.py:134] gen/LSTM/22/FC2 output: [200, 8]
[0929 16:14:32 @registry.py:126] gen/LSTM/22/FC3 input: [200, 8]
[0929 16:14:32 @registry.py:134] gen/LSTM/22/FC3 output: [200, 100]
[0929 16:14:32 @model.py:318] Creating cell for choice (in-edges: 7)
[0929 16:14:32 @registry.py:126] gen/LSTM/23/FC_inputs input: [200, 700]
[0929 16:14:32 @registry.py:134] gen/LSTM/23/FC_inputs output: [200, 100]
[0929 16:14:32 @registry.py:126] gen/LSTM/23/FC_attentions input: [

[0929 16:14:36 @base.py:209] Setup callbacks graph ...






[0929 16:14:37 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0929 16:14:37 @summary.py:93] Summarizing collection 'summaries' of size 9.

[0929 16:14:37 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.

[0929 16:14:37 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0929 16:14:39 @base.py:236] Initializing the session ...
[0929 16:14:39 @base.py:243] Graph Finalized.


[0929 16:14:39 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...

[0929 16:14:40 @base.py:275] Start Epoch 1 ...


 99%|################################################################################1|1980/2000[00:38<00:00,54.23it/s]

...


100%|#################################################################################|2000/2000[00:36<00:00,54.26it/s]

[0929 16:20:18 @base.py:285] Epoch 9 (global_step 18000) finished, time:36.9 seconds.
[0929 16:20:18 @saver.py:79] Model saved to ../output/Chicago/DATGAN_cpl/model\model-18000.
[0929 16:20:18 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.865
[0929 16:20:18 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.225
[0929 16:20:18 @monitor.py:467] GAN_loss/discrim/loss: 0.63525
[0929 16:20:18 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.1231
[0929 16:20:18 @monitor.py:467] GAN_loss/gen/klloss: 0.12235
[0929 16:20:18 @monitor.py:467] GAN_loss/gen/loss: 1.0008
[0929 16:20:18 @monitor.py:467] QueueInput/queue_size: 50
[0929 16:20:18 @base.py:275] Start Epoch 10 ...



100%|#################################################################################|2000/2000[00:36<00:00,54.09it/s]

[0929 16:20:55 @base.py:285] Epoch 10 (global_step 20000) finished, time:37 seconds.
[0929 16:20:55 @saver.py:79] Model saved to ../output/Chicago/DATGAN_cpl/model\model-20000.
[0929 16:20:55 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.91
[0929 16:20:55 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.21
[0929 16:20:55 @monitor.py:467] GAN_loss/discrim/loss: 0.64538
[0929 16:20:55 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0787
[0929 16:20:55 @monitor.py:467] GAN_loss/gen/klloss: 0.094512
[0929 16:20:55 @monitor.py:467] GAN_loss/gen/loss: 0.98415
[0929 16:20:55 @monitor.py:467] QueueInput/queue_size: 50
[0929 16:20:55 @base.py:289] Training has finished!

[0929 16:20:55 @model.py:318] Creating cell for age (in-edges: 0)


[0929 16:20:55 @model.py:318] Creating cell for gender (in-edges: 0)
[0929 16:20:55 @model.py:318] Creating cell for travel_dow (in-edges: 0)
[0929 16:20:55 @model.py:318] Creating cell for license (in-edges: 1)
[0929 16:20:55 @model.py:318] Creating cell for education_level (in-edges: 1)
[0929 16:20:55 @model.py:318] Creating cell for work_status (in-edges: 2)
[0929 16:20:55 @model.py:318] Creating cell for hh_income (in-edges: 2)
[0929 16:20:55 @model.py:318] Creating cell for trip_purpose (in-edges: 1)
[0929 16:20:55 @model.py:318] Creating cell for hh_descr (in-edges: 2)
[0929 16:20:55 @model.py:318] Creating cell for hh_size (in-edges: 3)
[0929 16:20:56 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[0929 16:20:56 @model.py:318] Creating cell for departure_time (in-edges: 1)
[0929 16:20:56 @model.py:318] Creating cell for distance (in-edges: 1)
[0929 16:20:56 @model.py:318] Creating cell for hh_vehicles (in-edges: 2)
[0929 16:20:56 @model.py:318] Creating cell 

In [12]:
datgan.save('trained', force=True)

[0929 16:21:04 @model.py:1109] Model saved successfully.
